# Data Preprocessing


In [6]:
import pandas as pd
from fastai.vision.all import *
import os

## Labeling the Datasets
Read the name of the file to get the number of fingers in the image. For example, "*_0L.png" means the image has 0 finger and left hand, "1R.png" means the image has 1 finger and right hand, and so on.

In [13]:
# Label the images
def label_func(f): return f.split('_')[1][0]

## Data Storage


In [15]:
data_path = '../data/fingers/'

# Create a dataframe with the image names and labels
train_file_list = os.listdir(os.path.join(data_path, 'train'))
train_data = pd.DataFrame({'file': train_file_list, 'fingers': [label_func(f) for f in train_file_list]})

test_file_list = os.listdir(os.path.join(data_path, 'test'))
test_data = pd.DataFrame({'file': test_file_list, 'fingers': [label_func(f) for f in test_file_list]})
